# Pyne

In [ ]:
import numpy as np
import os, sys, yaml, re
from IPython.display import display, Markdown, Latex #Can write latex too!!!!
import pandas as pd

import argparse
import shutil
import math
import matplotlib as mpl

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import scipy.optimize as so
import tables as tb

colors = plt.rcParams['axes.prop_cycle'].by_key()['color'];

#to view entire table (columns and rows)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)  

from pyne import ensdf, nucname, nuc_data, data

Declaring classes and parameters

In [ ]:
def GetQ(E, m):
    return np.multiply(E, (1 + np.divide(4.,m-4)));

class ADecay:
    def __init__(self, data):
        if isinstance(data[0], int):
            self.mother_id = int(data[0])
            #print('Setting mother_id = ', self.mother_id)
            if self.mother_id < 0:
                self.mother_id = -self.mother_id
        else:
            self.mother_id = None
        if isinstance(data[1], int):
            self.child_id = int(data[1])            
            #print('Setting child_id = ', self.child_id)
            if self.child_id < 0:
                self.child_id = -self.child_id
        else:
            self.child_id = None
        if data[3] is not None:
            self.half_life = float(data[3])
        else:
            self.half_life = np.nan
        if data[4] is not None:
            self.half_life_error = float(data[4])
        else:
            self.half_life_error = np.nan
        if data[5] is not None:
            self.branch_ratio = float(data[5])
        else:
            self.branch_ratio = np.nan
        if data[6] is not None:
            self.branch_ratio_error = float(data[6])
        else:
            self.branch_ratio_error = np.nan

        if self.mother_id is not None:
            self.A = int(nucname.anum(self.mother_id))
            self.Z = int(nucname.znum(self.mother_id))
            self.name = nucname.name(self.mother_id)
            self.Zname = nucname.zz_name[self.Z]
        else:
            #print('data[0] = ', data[0])
            rec = ensdf_mod.extra_SHE.match(data[0])
            self.A = int(rec.group(1))
            self.Z = int(ensdf_mod.d_extra_SHE[rec.group(2)])
            self.name = rec.group(2)+rec.group(1)
            self.Zname = rec.group(2)
        if self.child_id is not None:
            self.a = int(nucname.anum(self.child_id))
            self.z = int(nucname.znum(self.child_id))
        else:
            #print('data[1] = ', data[1])
            self.a = int(data[1][1])
            self.z = int(ensdf_mod.d_extra_SHE[data[1][0]])
        
        
        self.decay_mode = ''
        if self.A - self.a == 4:
            self.decay_mode = 'Alpha'
        elif self.A == self.a and self.Z == self.z+1:
            self.decay_mode = 'Beta_minus'
        elif self.A == self.a and self.Z == self.z-1:
            self.decay_mode = 'Beta_plus_or_EC'
        elif self.A == self.a and self.Z == self.z:
            self.decay_mode = 'gamma'
        #else:
            #print("WARNING NO DECAY MODE FOR", self)
        
        if self.name[-1] == 'M':
            self.isomer = True
        else: 
            self.isomer = False
        
        if data[-3] is not None:
            self.e_parent = data[-3]*1e-3
        else:
            self.e_parent = np.nan
        if self.e_parent > 0 and ~self.isomer:
            self.isomer = True
            print('WARNING: by pyne wrongly assigned isomer!', self.name)
                
        if data[-2] is not None:
            self.l_parent, self.l_parent_tent = data[-2]
        else:
            self.l_parent, self.l_parent_tent = np.nan, np.nan
        if data[-1] is not None:
            self.parity_parent, self.parity_parent_tent = data[-1]
        else:
            self.parity_parent, self.parity_parent_tent = np.nan, np.nan

        #print("parent vals: ", self.l_parent, self.l_parent_tent, self.parity_parent, self.parity_parent_tent)

        self.nbranches = 0
        self.E_alphas = None
        self.I_alphas = None
        self.child_levels = None
        self.child_l = None
        self.child_l_tent = None
        self.l_alphas = None
        self.l_alphas_tent = None
        self.child_parity = None
        self.child_parity_tent = None
        print(data[12])
        if data[12] is not None:
            self.nbranches = len(data[12])
            self.E_alphas = np.full(self.nbranches, np.nan)
            self.Q_alphas = np.full(self.nbranches, np.nan)
            self.I_alphas = np.full(self.nbranches, np.nan)
            self.child_levels = np.full(self.nbranches, np.nan)
            self.child_l = np.full(self.nbranches, np.nan)
            self.l_alphas = np.full(self.nbranches, np.nan)
            self.l_alphas_tent = np.full(self.nbranches, np.nan)
            self.child_l_tent = np.full(self.nbranches, np.nan)
            self.child_parity = np.full(self.nbranches, np.nan)
            self.child_parity_tent = np.full(self.nbranches, np.nan)
            for i, v in enumerate(data[12]):
                self.E_alphas[i] = v[2]
                self.Q_alphas[i] = GetQ(self.E_alphas[i]*.001, self.A)
                if v[3] is not None:
                    self.I_alphas[i] = v[3]*1e-2
                if v[4] is not None:
                    self.child_levels[i] = v[4]*1e-3
                if v[5] is not None:
                    self.child_l[i], self.child_l_tent[i] = v[5]
                if v[6] is not None:
                    self.child_parity[i], self.child_parity_tent[i] = v[6]
                self.l_alphas[i] = np.abs(self.l_parent-self.child_l[i])
                if any(np.isnan([self.child_l[i], self.l_parent])):
                    self.l_alphas_tent[i] = np.nan
                else:
                    self.l_alphas_tent[i] = bool(self.l_parent_tent)|bool(self.child_l_tent[i])
        
        #sometimes if only one branch, the intensity is not indicated at ensdf
        if self.nbranches == 1 and np.isnan(self.I_alphas[0]):
            self.I_alphas[0] = 1

            
     #overload string representation which is e.g. invoked in print()
    def __str__(self):
        string = self.name + ':\n'
        string += 'half-life = ' + str(self.half_life) + '+-' + str(self.half_life_error) + '\n'
        string += 'branch_ratio = ' + str(self.branch_ratio) + '+-' + str(self.branch_ratio_error) + '\n'
        string += "Q_alphas = " + np.str(self.Q_alphas) + '\n'
        string += "I_alphas = " + np.str(self.I_alphas) + '\n'
        string += "child_l = " + np.str(self.child_l)+ '\n'
        string += "child_l_tent = " + np.str(self.child_l_tent)+ '\n'
        string += "e_parent = " + np.str(self.e_parent)+ '\n'
        string += "l_parent = " + np.str(self.l_parent)+ '\n'
        string += "l_parent_tent = " + np.str(self.l_parent_tent)+ '\n'
        string += "l_alpha = " + np.str(self.l_alphas)+ '\n'
        string += "l_alpha_tent = " + np.str(self.l_alphas_tent)+ '\n'
        return string
        
    #overload representation which is important if the object is retrieved from list
    def __repr__(self):
        return str(self)

Read nuclear data and import into variables

In [ ]:
ind_names = ["Mother Nucleus"]

column_names = ["A", "Z", "Isomer?", "Q_alpha (MeV)", "t1/2 (s)", "I_alpha (%)", "I_alpha_branch (%)",  "Excitation energy (MeV)", "ang_mom_alpha", 'ang_mom_alpha_tent']

In [ ]:
alpha_list = ensdf.decays(filename='ensdf_alpha_decay.txt')
alpha_list = ensdf.decays(filename='ensdf_alpha_SHN.txt', decaylist=alpha_list)

In [ ]:
alpha_data = []
index = np.asarray([])
for i, v in enumerate(alpha_list):
    try:
        a = ADecay(v)
        if a.name == '':
            print('\ndata = ', v, '\n')
            print(a, "\n")
            break
        if a.decay_mode == 'Alpha':
            for j in range(a.nbranches):
                alpha_data.append([int(a.A), int(a.Z), int(a.isomer), float(a.Q_alphas[j]), float(a.half_life), float(a.branch_ratio), float(a.I_alphas[j]), float(a.child_levels[j]), float(a.l_alphas[j]), float(a.l_alphas_tent[j])])
                index = np.append(index, np.asarray([a.name]))
    except RuntimeError:
        print('RuntimeError: Nucleus not found!')

print('Shape index, data = ', np.shape(index.reshape(len(index), 1)), np.shape(np.asarray(alpha_data)))
index = index.reshape(len(index), 1)
alpha_data = np.asarray(alpha_data)
#print(alpha_data)
index = pd.MultiIndex.from_arrays(index.T, names=ind_names)

In [ ]:
alpha_data_table = dict(zip(column_names, alpha_data.T))
df = pd.DataFrame(data=alpha_data_table, index=index, columns=column_names)
#to view entire table (columns and rows)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
for i in range(3, len(df.columns)):
    df.iloc[:,i] = pd.to_numeric(df.iloc[:,i])
display(df)

In [ ]:
df['t1/2_alpha (s)'] = np.divide(df['t1/2 (s)'], df['I_alpha (%)'])
df['t1/2_alpha_part (s)'] = np.divide(df['t1/2_alpha (s)'], df['I_alpha_branch (%)'])
display(df)

In [ ]:
display(df.reset_index())
df = df.reset_index()

## Data corrections

For at least three of SHN the $Q_{\alpha}$ is a factor of 1000 too low. Here it is corrected:

In [ ]:
display(df[df['Q_alpha (MeV)'] < 1]['Q_alpha (MeV)'])
df.loc[df['Q_alpha (MeV)'] < 1, 'Q_alpha (MeV)'] = df[df['Q_alpha (MeV)'] < 1]['Q_alpha (MeV)']*1e3

In [ ]:
#removing 17=ecAlpha decay, 38, 1128 = almost exact duplicate & 547 = Pb202 with unknown branching ratio
df = df.drop([17, 38, 547, 1128])

In [ ]:
display('Duplicates: ', df[df.duplicated(keep=False)]) #displaying which are removed
df = df.drop_duplicates(keep='first')

In [ ]:
display(df)

## Writing data to binary file

In [ ]:
df.to_pickle('AlphaDecaysDataFrame3.p')